In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import datetime
from geopy.geocoders import Nominatim
from sklearn import preprocessing 
warnings.filterwarnings('ignore') 

from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

In [3]:
# Train data
data = pd.read_csv('C:/Users/susum/Documents/year5_sem1/QBUS3820/Group_Assignment/final_version/preprocessing_test.csv')
pd.set_option('display.max_columns', len(data.columns))
data.head()

,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,city,zipcode,smart_location,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,security_deposit_perc,cleaning_fee_perc,guests_included,extra_people_perc,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,47803920,2015-10-30,"Willoughby, New South Wales, Australia",Missing,0.0,0.0,0,Willoughby,2.0,2.0,['phone'],0,"Willoughby, NSW, Australia",Willoughby,Willoughby,Willoughby,2068.0,"Willoughby, Australia",-33.80537,151.19969,1,Apartment,Entire home/apt,5,1.0,2.0,4.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",194.0,139.0,1,22.0,5,10,5,5,10,10,5.0,10.0,30,60,90,365,1,0,2018-12-16,2018-12-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,flexible,0,0,2,1,1,0,0.07
1,23588866,2014-11-10,"Sydney, New South Wales, Australia",within a day,50.0,50.0,0,Avalon,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1,"Avalon Beach, NSW, Australia",Avalon,Pittwater,Avalon Beach,2107.0,"Avalon Beach, Australia",-33.63950,151.32333,1,House,Entire home/apt,6,2.0,4.0,4.0,Real Bed,"{TV,Wifi,Kitchen,""Free parking on premises"",""P...",200.0,30.0,6,20.0,2,1125,2,2,1125,1125,2.0,1125.0,0,4,4,232,10,2,2017-04-17,2020-01-04,93.0,10.0,10.0,10.0,10.0,10.0,9.0,0,0,0,strict_14_with_grace_period,0,0,1,1,0,0,0.29
2,55640456,2016-01-22,"New South Wales, Australia",within a few hours,98.0,98.0,1,Newtown/Enmore,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",0,"Newtown, NSW, Australia",Newtown/Enmore,Marrickville,Newtown,2042.0,"Newtown, Australia",-33.90481,151.17814,1,House,Entire home/apt,4,1.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Free stre...",503.0,45.0,1,0.0,3,1125,3,3,1125,1125,3.0,1125.0,10,32,44,61,82,32,2016-12-14,2020-01-20,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,strict_14_with_grace_period,0,0,1,1,0,0,2.12
3,28659104,2015-03-02,"Bondi Beach, New South Wales, Australia",within a day,100.0,100.0,0,Bondi,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1,"Bondi, NSW, Australia",Bondi,Waverley,Bondi,2026.0,"Bondi, Australia",-33.88985,151.26017,1,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,Kitchen,""Smoking allowed"",""Free stree...",0.0,0.0,1,0.0,2,14,2,2,14,14,2.0,14.0,0,0,0,0,135,0,2015-03-12,2018-11-11,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,strict_14_with_grace_period,0,0,2,1,1,0,2.25
4,9901656,2013-11-09,"Sydney, New South Wales, Australia",within an hour,86.0,86.0,1,Erskineville,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",0,"Erskineville, NSW, Australia",Erskineville,Sydney,Erskineville,2043.0,"Erskineville, Australia",-33.90001,151.18928,1,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Free street parking...",317.0,44.0,1,40.0,1,1125,1,1,1125,1125,1.0,1125.0,13,43,73,163,35,15,2014-11-25,2020-02-11,95.0,10.0,9.0,9.0,9.0,10.0,9.0,0,0,0,strict_14_with_grace_period,0,0,1,0,1,0,0.55


In [4]:
# Load keys for ammentities and host_verification
sorted_keys_v = np.genfromtxt('sorted_keys_v.csv',dtype='str')
sorted_keys_a = np.genfromtxt('sorted_keys_a.csv',dtype='str')
for i,w in enumerate(sorted_keys_a):
    sorted_keys_a[i] = w.replace("_"," ")

In [5]:
# Load keys for other dummies 
cancellation_policy = np.genfromtxt('cancellation_policy.csv',dtype='str')
neighbourhood_cleansed = np.genfromtxt('neighbourhood_cleansed.csv',dtype='str')
property_type = np.genfromtxt('property_type.csv',dtype='str')
room_type = np.genfromtxt('room_type.csv',dtype='str')
bed_type = np.genfromtxt('bed_type.csv',dtype='str')
host_response_time = np.genfromtxt('host_response_time.csv',dtype='str')

In [6]:
for i,w in enumerate(cancellation_policy):
    cancellation_policy[i] = w.replace("_"," ")
for i,w in enumerate(neighbourhood_cleansed):
    neighbourhood_cleansed[i] = w.replace("_"," ")
for i,w in enumerate(property_type):
    property_type[i] = w.replace("_"," ")
for i,w in enumerate(room_type):
    room_type[i] = w.replace("_"," ")   
for i,w in enumerate(bed_type):
    bed_type[i] = w.replace("_"," ")
for i,w in enumerate(host_response_time):
    host_response_time[i] = w.replace("_"," ")

In [19]:
removed_outliers = data['bedrooms'].between(data['bedrooms'].quantile(.05), data['bedrooms'].quantile(.95))
index_names = data[~removed_outliers].index
data.drop(index_names, inplace=True)
removed_outliers = data['beds'].between(data['beds'].quantile(.05), data['beds'].quantile(.95))
index_names = data[~removed_outliers].index
removed_outliers = data['bathrooms'].between(data['bathrooms'].quantile(.05), data['bathrooms'].quantile(.95))
index_names = data[~removed_outliers].index
data.drop(index_names, inplace=True)

# Replace date

In [7]:
# Use today as the baseline
# Replace the date by the number of days

today = datetime.date.today()
idx = data.columns.get_loc('host_since')

date_format = "%Y-%m-%d"
for i,date in enumerate(data.host_since):
    d = datetime.datetime.strptime(date,date_format).date()
    diff = today - d
    data.iloc[i,idx] = diff.days

In [8]:
# Use today as the baseline
# Replace the date by the number of days
# There are missing values in first and last review
# ignore if detected 

col = ['first_review','last_review']

for c in col:
    today = datetime.date.today()
    idx = data.columns.get_loc(c)

    date_format = "%Y-%m-%d"

    for i,date in enumerate(data[c]):
        try:
            d = datetime.datetime.strptime(date,date_format).date()
            diff = today - d
            data.iloc[i,idx] = diff.days
        except:
            print('NAN')

NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN


NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN


NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN


NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN


NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN


NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN
NAN


In [9]:
# Replace null values with 0 
to_fill = ['first_review',
          'last_review']
for i in to_fill:
    data[i] = data[i].fillna(0)

# Dummy Variables

In [10]:
# One hot encoding
to_encode = ['cancellation_policy',
             'neighbourhood_cleansed',
            'property_type',
            'room_type',
            'bed_type',
            'host_response_time']

# form comparison 
categorical= [cancellation_policy,
            neighbourhood_cleansed,
            property_type,
            room_type,
            bed_type,
            host_response_time]

# columns not in the training data
not_include = []

for i,j in enumerate(to_encode):
    
    # Find unique
    u = data[j].unique()
    
    # remove nan values 
    u  = [x for x in u if str(x) != 'nan']
    
    not_include.append(np.setdiff1d(u,categorical[i]))
    
    one_hot = pd.get_dummies(data[j])
    data = data.join(one_hot)
    
    # Drop original column 
    data = data.drop(j, axis = 1)

In [11]:
# Drop dublicated location info
cols_to_drop = ['luxury_moderate',
               'luxury_no_refund',
               'luxury_super_strict_125',
               'strict',
                'strict_14_with_grace_period',
                'super_strict_30',
                'super_strict_60',
                'Barn',
                'Bus',
                'Campsite',
                'Casa particular (Cuba)',
                'Cave',
                'Chalet',
                'Dome house',
                'Hut',
                'Plane',
                'Resort',
                'Tent',
                'Treehouse',
                'Airbed']
data = data.drop(cols_to_drop, axis = 1)

In [12]:
# One hot for host_verification
x = data['host_verifications']
x = [item.strip('[').strip(']').split(",") for item in x]
x = [subitem.lstrip().strip("'") for item in x for subitem in item]

from collections import Counter
count = Counter(x)

# include all 
selected = sorted_keys_v
var = np.zeros((len(data), len(sorted_keys_v)))
for i, info in enumerate(data['host_verifications']):
    for j, s in enumerate(selected):
        if s in info:
            var[i, j] = 1;
            
df = {}
for j, s in enumerate(selected):
    s = s.replace(" ","_")
    df[s+'_dummy'] = var[:, j]
    
df = pd.DataFrame(df)
del data['host_verifications']
data = data.join(df)

In [13]:
# One hot for amenities
x = data['amenities']
x = [item.strip("{").strip("}").replace('"', '').split(",") for item in x]
x = [word for item in x for word in item]

from collections import Counter
count = Counter(x)

# Pick all
selected = sorted_keys_a
var = np.zeros((len(data), len(sorted_keys_a)))
for i, info in enumerate(data['amenities']):
    for j, s in enumerate(selected):
        if s in info:
            var[i, j] = 1;
            
new_df = {}
for j, s in enumerate(selected):
    s = s.replace(" ","_")
    new_df[s+'_dummy'] = var[:, j]
    
new_df = pd.DataFrame(new_df)
del data['amenities']
data = data.join(new_df)

# Drop high collinearity items

In [15]:
cols_to_drop = ['host_acceptance_rate',
                'host_total_listings_count',
                'minimum_minimum_nights',
                'maximum_minimum_nights',
                'minimum_maximum_nights',
                'maximum_maximum_nights',
                'minimum_nights_avg_ntm',
                'maximum_nights_avg_ntm',
                'availability_30',
                'availability_60',
                'availability_365',
                'beds',
                'calculated_host_listings_count_private_rooms',
                'calculated_host_listings_count_shared_rooms',
                'number_of_reviews_ltm',
                'review_scores_accuracy',
                'review_scores_cleanliness', 'review_scores_checkin',
                'review_scores_communication', 'review_scores_location',
                'review_scores_value']
data = data.drop(cols_to_drop, axis = 1)

# Drop unrepresentative binary features

In [16]:
# Drop useless binary features 
cols_to_drop = [ 'requires_license',
                 'is_business_travel_ready',
                 'require_guest_profile_picture',
                 'require_guest_phone_verification']
data = data.drop(cols_to_drop, axis = 1)

In [17]:
# remove non ASCII features 
data.columns = data.columns.str.encode('ascii', 'ignore').str.decode('ascii')
data.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in data.columns]

# Drop duplicated location information

In [18]:
# Drop dublicated location info
cols_to_drop = ['city',
                'host_neighbourhood',
                'neighbourhood',
               'street',
               'smart_location',
               'host_location']
data = data.drop(cols_to_drop, axis = 1)

In [18]:
data = data.drop("host_id", axis = 1)
#Test data
data.to_csv(r'C:\Users\susum\Documents\year5_sem1\QBUS3820\Group_Assignment\final_version\test_2.csv',index = False, header = True)